In [69]:
df = pd.read_csv(base_path / 'Test_Cases.csv', index_col='Case')
df

,clinical_note
Case,
1,A 44-year- old super-morbidly- obese man body ...
2,A 32-year-old man presented to a regional gene...
3,"In December 1990, a 24-year-old female was ref..."
4,A 30-year-old man with a history of chronic ab...
5,An 85-year-old man was referred to rheumatolog...
...,...
112,A 14-year-old male patient presented with a 3-...
113,"A 15-year-old, otherwise healthy, Caucasian gi..."
114,A man in his late 40s came to our emergency de...


In [70]:
df.to_csv(base_path / 'cases_rag_hpo.csv', index=True)

In [86]:
import pandas as pd
from pathlib import Path
from utils.api_usage import get_relevant
from tqdm.auto import tqdm

base_path = Path().resolve() / 'files'

df = pd.read_csv(base_path / 'Test_Cases.csv', index_col='Case')

test_cases_relevant_terms = {}
test_cases_related_terms = {}
for i, case in tqdm(df.iterrows(), total=len(df)):
    print(f"Case {i}: {case.values[0]}\n")
    relevant_hpo_terms, related_hpo_terms = get_relevant(case.values[0])
    test_cases_relevant_terms[i] = relevant_hpo_terms
    test_cases_related_terms[i] = related_hpo_terms
    # print(f"Related HPO terms: {related_hpo_terms}")
    # print(f"Relevant HPO terms: {relevant_hpo_terms}\n")

  0%|          | 0/116 [00:00<?, ?it/s]

Case 1: A 44-year- old super-morbidly- obese man body mass index (BMI 63) underwent sleeve gastrectomy for weight loss and was found to have multiple adenomatous fundic gland polyps on final pathology. Subsequent workup included esophagogastroduodenoscopy which revealed innumerable polyps of the remaining gastric fundus and body consistent with fundic gland polyps, normal duodenum without polyps, and Barrett’s oesophagus. Colonoscopy was significant for innumerable polyps of varying sizes up to 1.5 cm throughout the colon, with relative rectal sparing. Biopsies were consistent with tubular adenoma and hyperplastic polyps. Thyroid ultrasound was within normal limits and abdominal CT was significant for left-sided 3.4 cm mesenteric mass representing scarring versus possible desmoid. Family history was significant for colon cancer diagnosed in his maternal grandfather at age 72, paternal grandmother who died of metastatic cancer at age 50 with unknown primary and mother diagnosed with thy

In [48]:
cases_results_df = pd.read_excel(base_path / 'RAG-HPO_Tests_and_Data_Analysis.xlsx', sheet_name='Cases Results', header=1)
cases_results_df = cases_results_df[cases_results_df['HPO ID'] != 'HPO ID']
cases_results_df

,Unnamed: 0,Phenotype name,HPO ID,Unnamed: 3,Phenotype name.1,HPO ID.1,Unnamed: 6,Phenotype name.2,HPO ID.2,Unnamed: 9,Phenotype name.3,HPO ID.3,Unnamed: 12,Phenotype name.4,HPO ID.4,Unnamed: 15,Phenotype name.5,HPO ID.5
0,11,Multicystic kidney dysplasia,HP:0000003,NaN,multicystic dysplastic kidney,HP:0000003,NaN,NaN,NaN,NaN,multicystic dysplastic kidney,HP:0000003,NaN,Multicystic kidney dysplasia,HP:0000003,NaN,Multicystic dysplastic kidney,HP:0000003
1,NaN,Decreased fetal movement,HP:0001558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,decreased fetal movement,HP:0001558,NaN,Decreased fetal movement,HP:0001558,NaN,decreased fetal movement,HP:0001558
2,NaN,Subglottic stenosis,HP:0001607,NaN,subglottic stenosis,HP:0001607,NaN,Grade I subglottic stenosis (40% stenosis,HP:0001156,NaN,subglottic stenosis,HP:0001607,NaN,Subglottic stenosis,HP:0001607,NaN,Subglottic stenosis,HP:0001607
3,NaN,Mild mitral regurgitation,HP:0001653,NaN,mitral regurgitation,HP:0001653,NaN,Mitral regurgitation,HP:0001654,NaN,mitral regurgitation,HP:0001653,NaN,Mitral regurgitation,HP:0001653,NaN,"Mitral regurgitation, mild",HP:0001653
4,NaN,Aphonia,HP:0001686,NaN,aphonia,HP:0001686,NaN,Aphonia,HP:0000395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aphonia,HP:0001686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3093,NaN,concerns about perceived body odor,HP:0034004,NaN,olfactory reference syndrome,HP:0034004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3094,NaN,avoidance behavior,HP:5200320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3095,NaN,overvalued idea,HP:5200402,NaN,overvalued idea,HP:5200402,NaN,NaN,NaN,NaN,overvalued idea,HP:5200402,NaN,NaN,NaN,NaN,NaN,NaN
3096,NaN,NaN,NaN,NaN,severe distress,HP:5200271,NaN,NaN,NaN,NaN,body odor,HP:0500001,NaN,body odor,HP:0500001,NaN,Body odor,HP:0500001


In [75]:
# Assuming 'HPO ID' is the column containing the HPO codes
column_name = "HPO ID"

# Add a unique patient ID by tracking NaNs
cases_results_df["Patient ID"] = (cases_results_df[column_name].isna()).cumsum()

# Drop NaNs since they are just separators
cleaned_df = cases_results_df.dropna(subset=[column_name])

# Group by Patient ID and aggregate HPO codes into a list
patients_dict = cleaned_df.groupby("Patient ID")[column_name].apply(list).to_dict()

# Convert to a DataFrame for easy viewing
patients_df = pd.DataFrame(list(patients_dict.items()), columns=["Patient ID", "HPO Codes"])
patients_df


,Patient ID,HPO Codes
0,0,"[HP:0000003, HP:0001558, HP:0001607, HP:000165..."
1,9,"[HP:0000476, HP:0000773, HP:0001591, HP:000265..."
2,17,"[HP:0000997, HP:0001067, HP:0002108, HP:000095..."
3,23,"[HP:0000286, HP:0000289, HP:0000384, HP:000041..."
4,30,"[HP:0000023, HP:0000218, HP:0000767, HP:000097..."
...,...,...
111,1094,"[HP:0000713, HP:0001289, HP:0001903, HP:000213..."
112,1105,"[HP:0000716, HP:0000742, HP:0000757, HP:000182..."
113,1114,"[HP:0000709, HP:0000711, HP:0000713, HP:000073..."
114,1119,"[HP:0000338, HP:0000508, HP:0000716, HP:000207..."


In [76]:
# Assuming 'HPO ID' is the column containing the HPO codes
column_name = "Phenotype name"

# Add a unique patient ID by tracking NaNs
cases_results_df["Patient ID"] = (cases_results_df[column_name].isna()).cumsum()

# Drop NaNs since they are just separators
cleaned_df = cases_results_df.dropna(subset=[column_name])

# Group by Patient ID and aggregate HPO codes into a list
patients_dict = cleaned_df.groupby("Patient ID")[column_name].apply(list).to_dict()

# Convert to a DataFrame for easy viewing
patients_df_phenotype = pd.DataFrame(list(patients_dict.items()), columns=["Patient ID", "Phenotype Names"])
patients_df_phenotype

,Patient ID,Phenotype Names
0,0,"[Multicystic kidney dysplasia, Decreased fetal..."
1,8,"[Manually Assigned HPO Terms, Cystic hygroma, ..."
2,15,"[Manually Assigned HPO Terms, Axillary freckli..."
3,20,"[Manually Assigned HPO Terms, Bilateral epican..."
4,26,"[Manually Assigned HPO Terms, Inguinal hernia,..."
...,...,...
111,976,"[Manually Assigned HPO Terms, Agitation , diso..."
112,986,"[Manually Assigned HPO Terms, major depressive..."
113,994,"[Manually Assigned HPO Terms, acute psychosis,..."
114,998,"[Manually Assigned HPO Terms, Facial dullness,..."


In [77]:
# Separate the first 20 rows (rows 0-20)
first_20_rows = patients_df.iloc[:20]  # Includes row 20

# Get the remaining rows (excluding the first 20)
remaining_rows = patients_df.iloc[20:]

# Concatenate them, placing the first 20 rows at the bottom
patients_df = pd.concat([remaining_rows, first_20_rows], ignore_index=True)

first_20_rows = patients_df_phenotype.iloc[:20]  # Includes row 20
remaining_rows = patients_df_phenotype.iloc[20:]
patients_df_phenotype = pd.concat([remaining_rows, first_20_rows], ignore_index=True)

In [80]:
patients_df['Patient ID'] = (patients_df.index + 1)
patients_df_phenotype['Patient ID'] = (patients_df.index + 1)
patients_df

,Patient ID,HPO Codes
0,1,"[HP:0001513, HP:0004394, HP:0005227, HP:001218..."
1,2,"[HP:0001903, HP:0003406, HP:0012185, HP:000439..."
2,3,"[HP:0000622, HP:0000853, HP:0003002, HP:000790..."
3,4,"[HP:0002019, HP:0002027, HP:0002035, HP:000257..."
4,5,"[HP:0002355, HP:0003216, HP:0003236, HP:000356..."
...,...,...
111,112,"[HP:0001433, HP:0001649, HP:0001744, HP:000182..."
112,113,"[HP:0000939, HP:0001254, HP:0001533, HP:000280..."
113,114,"[HP:0000822, HP:0001945, HP:0001974 , HP:0002..."
114,115,"[HP:0002066, HP:0002080, HP:0002172, HP:000217..."


In [79]:
patients_df_phenotype

,Patient ID,Phenotype Names
0,169,"[Manually Assigned HPO Terms, obesity , innume..."
1,175,"[Manually Assigned HPO Terms, anemia, C5 nerve..."
2,183,"[Manually Assigned HPO Terms, blurred vision, ..."
3,191,"[Manually Assigned HPO Terms, constipation, ch..."
4,195,"[Manually Assigned HPO Terms, walking difficul..."
...,...,...
111,109,"[Manually Assigned HPO Terms, hepatosplenomega..."
112,123,"[Manually Assigned HPO Terms, osteoporosis, le..."
113,138,"[Manually Assigned HPO Terms, hypertension , f..."
114,143,"[Manually Assigned HPO Terms, ataxia , intenti..."


In [99]:
import pandas as pd
import textwrap

# Convert the ground truth DataFrame to a dictionary for lookup
ground_truth_hpo_dict = dict(zip(patients_df["Patient ID"], patients_df["HPO Codes"]))
ground_truth_phenotype_dict = dict(zip(patients_df_phenotype["Patient ID"], patients_df_phenotype["Phenotype Names"]))

# Lists to store per-patient scores
accuracies = []
precisions = []
recalls = []

tps = [] 
fps = []
fns = []
count = 0 
# Iterate through each patient in the predicted dictionary
for patient_id, predicted_hpo_terms in test_cases_relevant_terms.items():

    true_hpo_terms = set(ground_truth_hpo_dict.get(patient_id, []))  # Get true HPO terms from patients_df
    predicted_hpo_terms = set(predicted_hpo_terms)  # Convert to set for comparison
    true_phenotype_terms = set(ground_truth_phenotype_dict.get(patient_id, []))

    # Wrap the case description
    try:
        wrapped_text = textwrap.fill(df.iloc[patient_id-1]['clinical_note'], width=120)

        print(f"Patient ID: {patient_id}")
        print(wrapped_text)
        print("\n")
    except:
        print(f"Patient ID: {patient_id}")
        continue
    
    try: 
        true_phenotype_terms.remove('Manually Assigned HPO Terms')
    except:
        continue

    print(f"RAG-HPO: {true_phenotype_terms}")
    print(f"ChatIMD: {list(test_cases_relevant_terms[patient_id].values())}")
    print(f"RAG-HPO: {true_hpo_terms}")
    print(f"ChatIMD: {predicted_hpo_terms}")
    print("\n")

    # Calculate metrics
    tp = len(predicted_hpo_terms & true_hpo_terms)  # True Positives (correctly predicted terms)
    fp = len(predicted_hpo_terms - true_hpo_terms)  # False Positives (incorrect predictions)
    fn = len(true_hpo_terms - predicted_hpo_terms)  # False Negatives (missed correct terms)

    if tp == 0:
        count += 1

    tps.append(tp)
    fps.append(fp)
    fns.append(fn)

    print(f"True Positives (TP): {predicted_hpo_terms & true_hpo_terms}")
    print(f"False Positives (FP): {predicted_hpo_terms - true_hpo_terms}")
    print(f"False Negatives (FN): {true_hpo_terms - predicted_hpo_terms}")

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    accuracy = tp / len(true_hpo_terms) if len(true_hpo_terms) > 0 else 0

    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")  
    print(f"Accuracy: {accuracy:.2f}\n")

    precisions.append(precision)
    recalls.append(recall)
    accuracies.append(accuracy)

# Compute overall averages
overall_precision = sum(precisions) / len(precisions) if precisions else 0
overall_recall = sum(recalls) / len(recalls) if recalls else 0
overall_accuracy = sum(accuracies) / len(accuracies) if accuracies else 0
overall_f1_score = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

# Display results
results_df = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1 Score", "TP", "FP", "FN"],
    "Score": [round(overall_accuracy, 2), round(overall_precision, 2), round(overall_recall, 2), round(overall_f1_score, 2), int(sum(tps)), int(sum(fps)), int(sum(fns))]
})

print(f"Count: {count}")
results_df

Patient ID: 1
A 44-year- old super-morbidly- obese man body mass index (BMI 63) underwent sleeve gastrectomy for weight loss and was
found to have multiple adenomatous fundic gland polyps on final pathology. Subsequent workup included
esophagogastroduodenoscopy which revealed innumerable polyps of the remaining gastric fundus and body consistent with
fundic gland polyps, normal duodenum without polyps, and Barrett’s oesophagus. Colonoscopy was significant for
innumerable polyps of varying sizes up to 1.5 cm throughout the colon, with relative rectal sparing. Biopsies were
consistent with tubular adenoma and hyperplastic polyps. Thyroid ultrasound was within normal limits and abdominal CT
was significant for left-sided 3.4 cm mesenteric mass representing scarring versus possible desmoid. Family history was
significant for colon cancer diagnosed in his maternal grandfather at age 72, paternal grandmother who died of
metastatic cancer at age 50 with unknown primary and mother diagnosed wi

,Metric,Score
0,Accuracy,0.50
1,Precision,0.57
2,Recall,0.50
3,F1 Score,0.53
4,TP,921.00
5,FP,628.00
6,FN,904.00
